In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [12]:

# Fetch DAX data
ticker_symbol = "^GDAXI"
dax_data = yf.download(ticker_symbol, start="1900-01-01", end="2025-01-13")

# Convert to a Pandas DataFrame
dax_df = pd.DataFrame(dax_data)

# Bundestagswahl data
elections_data = [
    ("1949-08-14", "CDU/CSU, FDP, DP"),
    ("1953-09-06", "CDU/CSU, FDP, DP, GB/BHE"),
    ("1957-09-15", "CDU/CSU, DP"),
    ("1961-09-17", "CDU/CSU, FDP"),
    ("1965-09-19", "CDU/CSU, SPD"),
    ("1969-09-28", "SPD, FDP"),
    ("1972-11-19", "SPD, FDP"),
    ("1976-10-03", "SPD, FDP"),
    ("1980-10-05", "SPD, FDP"),
    ("1983-03-06", "CDU/CSU, FDP"),
    ("1987-01-25", "CDU/CSU, FDP"),
    ("1990-12-02", "CDU/CSU, FDP"),
    ("1994-10-16", "CDU/CSU, FDP"),
    ("1998-09-27", "SPD, Greens"),
    ("2002-09-22", "SPD, Greens"),
    ("2005-09-18", "CDU/CSU, SPD"),
    ("2009-09-27", "CDU/CSU, FDP"),
    ("2013-09-22", "CDU/CSU, SPD"),
    ("2017-09-24", "CDU/CSU, SPD"),
    ("2021-09-26", "SPD, Greens, FDP")
]

[*********************100%***********************]  1 of 1 completed


In [18]:
def calculate_performance(date):
    election_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = election_date - timedelta(weeks=6)
    end_date = election_date + timedelta(weeks=6)
    
    # Ensure we have data in the specified range
    if start_date < dax_df.index[0]:
        return None, None
    
    # Filter the DAX data for the relevant period
    period_data = dax_df[(dax_df.index >= start_date) & (dax_df.index <= end_date)]
    
    if not period_data.empty:
        start_price = period_data['Close'].iloc[0]
        
        # Find the closest date to the election date within the period
        nearest_date = period_data.index[np.abs(period_data.index - election_date).argmin()]
        mid_price = period_data['Close'].loc[nearest_date]
        
        end_price = period_data['Close'].iloc[-1]
        
        before_performance = ((mid_price / start_price) - 1) * 100
        after_performance = ((end_price / mid_price) - 1) * 100
        
        return round(before_performance, 2), round(after_performance, 2)
    
    return None, None

# Calculate performance for each election and store results
performances = []
for date, parties in elections_data:
    before, after = calculate_performance(date)
    
    # Append results ensuring numerical values are stored directly
    performances.append({
        'Election Date': date,
        'Governing Parties': parties,
        'Performance Before (%)': before,
        'Performance After (%)': after
    })

# Create DataFrame with election results and DAX performance
election_performance_df = pd.DataFrame(performances)

# Clean up performance columns to ensure they are numeric and handle NaN values properly
election_performance_df['Performance Before (%)'] = election_performance_df['Performance Before (%)'].astype(float)
election_performance_df['Performance After (%)'] = election_performance_df['Performance After (%)'].astype(float)

print(election_performance_df.tail(10))

   Election Date Governing Parties  Performance Before (%)  \
10    1987-01-25      CDU/CSU, FDP                     NaN   
11    1990-12-02      CDU/CSU, FDP                   -0.53   
12    1994-10-16      CDU/CSU, FDP                   -3.59   
13    1998-09-27       SPD, Greens                  -13.89   
14    2002-09-22       SPD, Greens                  -20.09   
15    2005-09-18      CDU/CSU, SPD                    1.82   
16    2009-09-27      CDU/CSU, FDP                   10.28   
17    2013-09-22      CDU/CSU, SPD                    3.30   
18    2017-09-24      CDU/CSU, SPD                    3.53   
19    2021-09-26  SPD, Greens, FDP                   -2.21   

    Performance After (%)  
10                    NaN  
11                  -6.06  
12                  -1.88  
13                   2.83  
14                   8.61  
15                  -2.04  
16                  -4.32  
17                   4.31  
18                   7.02  
19                   3.09  


C:\Users\softw\AppData\Local\Temp\ipykernel_11932\2058339481.py:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['Performance Before (%)'] = election_performance_df['Performance Before (%)'].astype(float)
C:\Users\softw\AppData\Local\Temp\ipykernel_11932\2058339481.py:47: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['Performance After (%)'] = election_performance_df['Performance After (%)'].astype(float)


In [19]:
# Save the election performance data to a CSV file
election_performance_df.to_csv("data/election_dax_performance.csv", index=False)

In [20]:
# Calculate weekly returns and average weekly growth
dax_df['Weekly Return'] = dax_df['Close'].pct_change()  # Calculate weekly percentage change

# Group by week and calculate average weekly return
average_weekly_growth = dax_df['Weekly Return'].resample('W').mean() * 100  # Convert to percentage

# Calculate overall average weekly growth
overall_average_weekly_growth = average_weekly_growth.mean()

print(f"Average Weekly Growth of DAX: {overall_average_weekly_growth:.2f}%")

Average Weekly Growth of DAX: 0.05%


In [34]:
def calculate_performance(date):
    election_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = election_date - timedelta(weeks=1)
    end_date = election_date + timedelta(weeks=12)
    
    # Ensure we have data in the specified range
    if start_date < dax_df.index[0]:
        return None, None
    
    # Filter the DAX data for the relevant period
    period_data = dax_df[(dax_df.index >= start_date) & (dax_df.index <= end_date)]
    
    if not period_data.empty:
        start_price = period_data['Close'].iloc[0]
        
        # Find the closest date to the election date within the period
        nearest_date = period_data.index[np.abs(period_data.index - election_date).argmin()]
        mid_price = period_data['Close'].loc[nearest_date]
        
        end_price = period_data['Close'].iloc[-1]
        
        return round(start_price, 2), round(end_price, 2)
    
    return None, None

# Calculate performance for each election and store results
performances = []
for date, parties in elections_data:
    before, after = calculate_performance(date)
    
    # Append results ensuring numerical values are stored directly
    performances.append({
        'Election Date': date,
        'Governing Parties': parties,
        'start_price Before (%)': before,
        'end_price After (%)': after
    })

# Create DataFrame with election results and DAX performance
election_performance_df = pd.DataFrame(performances)

# Clean up performance columns to ensure they are numeric and handle NaN values properly
election_performance_df['start_price Before (%)'] = election_performance_df['start_price Before (%)'].astype(float)
election_performance_df['end_price After (%)'] = election_performance_df['end_price After (%)'].astype(float)

print(election_performance_df.tail(10))

   Election Date Governing Parties  start_price Before (%)  \
10    1987-01-25      CDU/CSU, FDP                     NaN   
11    1990-12-02      CDU/CSU, FDP                 1458.28   
12    1994-10-16      CDU/CSU, FDP                 2048.56   
13    1998-09-27       SPD, Greens                 4439.13   
14    2002-09-22       SPD, Greens                 3319.05   
15    2005-09-18      CDU/CSU, SPD                 4989.98   
16    2009-09-27      CDU/CSU, FDP                 5668.65   
17    2013-09-22      CDU/CSU, SPD                 8613.00   
18    2017-09-24      CDU/CSU, SPD                12559.39   
19    2021-09-26  SPD, Greens, FDP                15132.06   

    end_price After (%)  
10                  NaN  
11              1574.77  
12              2058.20  
13              4666.74  
14              3077.06  
15              5282.13  
16              5831.21  
17              9006.46  
18             13103.56  
19             15531.69  


C:\Users\softw\AppData\Local\Temp\ipykernel_11932\3703343756.py:43: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['start_price Before (%)'] = election_performance_df['start_price Before (%)'].astype(float)
C:\Users\softw\AppData\Local\Temp\ipykernel_11932\3703343756.py:44: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['end_price After (%)'] = election_performance_df['end_price After (%)'].astype(float)


In [35]:
# Save the election performance data to a CSV file
election_performance_df.to_csv("data/election_dax_performance.csv", index=False)

: 